In [1]:
from pymongo import MongoClient
import pandas as pd
from src.functions import * 
from src.df_computation import *
from src.strbkcs_computation import *
from src.schls_computation import *
from src.brs_computation import *
from src.rpts_computation import *
from src.vgn_computation import *
from keplergl import KeplerGl
from pandas.io.json import json_normalize

In [2]:
dbName = "companies"
mongodbURL = f"mongodb://localhost/{dbName}"
client = MongoClient(mongodbURL, connectTimeoutMS=2000,serverSelectionTimeoutMS=2000)
db = client.get_database()

# Indentify Old Companies (>10y)

In [3]:
old_comps = list(db.companies.find({"founded_year":{"$lte":2003}},{"_id":0,"name":1,"category_code":1,"total_money_raised":1,"founded_year":1,"offices.city":1,"offices.country_code":1,"offices.latitude":1,"offices.longitude":1}))
dfo = normalize(old_comps)

In [4]:
dfo = dfo.dropna(subset=["latitude"])

In [5]:
dfo["coordinates"] = dfo.apply(lambda row:{"type":"Point", "coordinates":[row.latitude,row.longitude]},axis=1)

In [6]:
export_dfo = dfo.to_csv ('OUTPUT/dfo.csv', index=None, header=True)

# Indentify 1M Tech Startups

In [7]:
cool_comps = list(db.companies.find({"$and":[{"founded_year":{"$gt":2003}},{"total_money_raised":{"$not":{"$regex":"\$0|\.*k"}}}]},{"_id":0,"name":1,"category_code":1,"total_money_raised":1,"founded_year":1,"offices.city":1,"offices.country_code":1,"offices.latitude":1,"offices.longitude":1}))
dfc = normalize(cool_comps)

In [8]:
dfc = dfc.dropna(subset=['latitude'])

In [9]:
dfc = dfc.loc[dfc.category_code.isin(tech_comps)]

In [10]:
dfc["coordinates"] = dfc.apply(lambda row:{"type":"Point", "coordinates":[row.latitude,row.longitude]},axis=1)

In [11]:
export_dfc = dfc.to_csv ('OUTPUT/dfc.csv', index=None, header=True)

# Select Tech Startups out of Old Companies Radius

In [12]:
dfc["check"] = dfc.apply(checkCoord,axis=1)

In [13]:
df_clean = dfc.loc[dfc.check=="outside"]

In [14]:
export_df_clean = df_clean.to_csv('OUTPUT/df_clean.csv', index=None, header=True)

# Find Starbucks

In [15]:
df_clean["starbucks"] = df_clean.apply(getStarbucks,axis=1)

/home/diego/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [16]:
df_clean["starbucks_count"] = df_clean.apply(countStarbucks,axis=1)

/home/diego/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


# Find Schools

In [17]:
df_clean["schools"] = df_clean.apply(getSchools,axis=1)

/home/diego/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [18]:
df_clean["schools_count"] = df_clean.apply(countSchools,axis=1)

/home/diego/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


# Find Bars

In [19]:
df_clean["bars"] = df_clean.apply(getBars,axis=1)

/home/diego/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [20]:
df_clean["bars_count"] = df_clean.apply(countBars,axis=1)

/home/diego/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


# Find Airports

In [21]:
df_clean["airports"] = df_clean.apply(getAirports,axis=1)

/home/diego/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [22]:
df_clean["airports_count"] = df_clean.apply(countAirports,axis=1)

/home/diego/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


# Find Vegan Restaurants

In [23]:
df_clean["vegan"] = df_clean.apply(getVegan,axis=1)

/home/diego/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [24]:
df_clean["vegan_count"] = df_clean.apply(countVegan,axis=1)

/home/diego/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


# Final DF

In [25]:
df_clean

,city,country_code,latitude,longitude,name,category_code,total_money_raised,founded_year,coordinates,check,starbucks,starbucks_count,schools,schools_count,bars,bars_count,airports,airports_count,vegan,vegan_count
173,None,ISR,17.088810,-96.764920,Sportingo,web,$3.2M,2006,"{'type': 'Point', 'coordinates': [17.08881, -9...",outside,no,0,"[New Mexico High School General, {'lat': 17.08...",20,"[La Rinconada, {'lat': 17.0957441, 'lng': -96....",15,"[Taxi Tour Valle de Oaxaca Antequera, {'lat': ...",2,"[Annona Pan Integral, {'lat': 17.0931677, 'lng...",2
738,TromsÃ¸,NOR,69.636216,18.931062,Playfish,games_video,$21M,2007,"{'type': 'Point', 'coordinates': [69.636216, 1...",outside,no,0,"[Kongsbakken Upper Secondary School, {'lat': 6...",15,"[Rorbua Pub, {'lat': 69.6489929, 'lng': 18.961...",20,"[The Arctic Route, {'lat': 69.6486448, 'lng': ...",13,"[Tromso Kaffebrenneri - Veita, {'lat': 69.6519...",20
1171,Dhanmondi,BGD,23.743156,90.379976,Code71,software,$1M,2006,"{'type': 'Point', 'coordinates': [23.743156, 9...",outside,no,0,"[Mohammadpur Preparatory School & College, {'l...",20,"[Chalet Bar & Restaurant, {'lat': 23.7444123, ...",20,"[Helipad, {'lat': 23.7541347, 'lng': 90.400074...",4,"[Santoor Restaurant, {'lat': 23.7518678, 'lng'...",14
1247,Saskatoon,CAN,52.143886,-106.663456,MyFrontSteps,web,$3M,2008,"{'type': 'Point', 'coordinates': [52.143886, -...",outside,"[Starbucks, {'lat': 52.1315654, 'lng': -106.64...",6,"[E. D. Feehan Catholic High School, {'lat': 52...",18,"[Hudsons Saskatoon, {'lat': 52.1263771, 'lng':...",20,no,0,"[The Karma - Conscious Café & Eatery, {'lat': ...",20
1663,Tartu,USA,58.382054,26.731054,ZeroTurnaround,software,$2.7M,2007,"{'type': 'Point', 'coordinates': [58.3820542, ...",outside,no,0,"[Tartu International School MTÜ, {'lat': 58.37...",20,"[Illegaard, {'lat': 58.379455, 'lng': 26.72237...",20,"[VIRTA Charging Station, {'lat': 58.377787, 'l...",9,"[Krempel Kohvik, {'lat': 58.3815493, 'lng': 26...",20
1973,Tartu,EST,58.375411,26.721221,Fortumo,mobile,$10M,2007,"{'type': 'Point', 'coordinates': [58.3754107, ...",outside,no,0,"[Tartu International School MTÜ, {'lat': 58.37...",20,"[Illegaard, {'lat': 58.379455, 'lng': 26.72237...",20,"[VIRTA Charging Station, {'lat': 58.377787, 'l...",9,"[Krempel Kohvik, {'lat': 58.3815493, 'lng': 26...",20
2058,"Abertillery, Gwent,",GBR,23.066066,-82.292780,Atraverda,cleantech,£16.6M,2005,"{'type': 'Point', 'coordinates': [23.066066, -...",outside,no,0,no,0,"[Pilar, {'lat': 23.0677356, 'lng': -82.2951503}]",1,no,0,"[McCesar's, {'lat': 23.0518415, 'lng': -82.275...",1
2136,Perth,AUS,-31.978220,115.890840,HealthEngine,web,$10.4M,2007,"{'type': 'Point', 'coordinates': [-31.97822, 1...",outside,no,0,"[St Clare's School, {'lat': -31.9712183, 'lng'...",20,"[Plain Street Bar, {'lat': -31.96118019999999,...",20,"[Find Airport Transfer, {'lat': -31.9604702, '...",1,"[Loving Hut Victoria Park, {'lat': -31.9738044...",17
2522,Saskatoon,CAN,52.143886,-106.663456,VendAsta,software,C$11.3M,2008,"{'type': 'Point', 'coordinates': [52.143886, -...",outside,"[Starbucks, {'lat': 52.1315654, 'lng': -106.64...",6,"[E. D. Feehan Catholic High School, {'lat': 52...",18,"[Hudsons Saskatoon, {'lat': 52.1263771, 'lng':...",20,no,0,"[The Karma - Conscious Café & Eatery, {'lat': ...",20


In [26]:
export_df_clean = df_clean.to_csv('OUTPUT/df_final.csv', index=None, header=True)

Once you have seen the different options, choose the perfect location for you
hint: location = rowIndex

In [27]:
row_index = input("Select the row: ")

Select the row: 3


In [28]:
#cleaning places
index = int(row_index)

starbucks = [e for e in df_clean.starbucks][index]
schools = [e for e in df_clean.schools][index]
bars = [e for e in df_clean.bars][index]
stations = [e for e in df_clean.airports][index]
vegan = [e for e in df_clean.vegan][index]

In [29]:
#getting correct final coords
coords_ok = df_clean.iloc[[index],[0,2,3]]

starbucks_ok = starbucksOK(starbucks)
schools_ok = schoolsOK(schools)
bars_ok = barsOK(bars)
stations_ok = stationsOK(stations)
vegan_ok = veganOK(vegan)

# Map

In [30]:
#loading map config
%run src/hex_config.py

map_1 = KeplerGl(height=500, config=config)
map_1.add_data(data=coords_ok, name="Final Location")

map_1.add_data(data=starbucks_ok, name="starbucks")
map_1.add_data(data=schools_ok, name="schools")
map_1.add_data(data=bars_ok, name="bars")
map_1.add_data(data=stations_ok, name="stations")
map_1.add_data(data=vegan_ok, name="vegan")

map_1

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': '3kjv467', 'type': …

In [31]:
#saving map config
config = map_1.config
with open('src/hex_config.py', 'w') as f:
    f.write('config = {}'.format(map_1.config))

In [32]:
#saving map as html
map_1.save_to_html(config=config, file_name='OUTPUT/first_map.html')

Map saved to OUTPUT/first_map.html!
